I'm going to automate my first pass data GPT-4 generation using the API. 

In [8]:
from pathlib import Path
import os
from openai import OpenAI
from dotenv import load_dotenv
import random
import time

load_dotenv()

Path.ls = lambda x: list(x.iterdir())

In [4]:
ASST_ID = os.environ.get("ASST_ID")

PROJECT = Path("../../")

PROMPT = PROJECT / "prompt.txt"

DATA = PROJECT / "data"
len(DATA.ls())

1

In [6]:
example = random.choice(DATA.ls())
fp = example / "recipe.txt"
recipe = fp.read_text()
print(recipe)

Ingredients
1 tbsp ponzu sauce
½ tbsp rice vinegar
5g ginger, peeled and grated
1 tsp sesame oil
300g silken tofu
100g edamame beans
250g pouch cooked quinoa
100g radishes, sliced
2 carrots, peeled into ribbons
2 spring onions, finely sliced
2 small seaweed thins, crumbled
1 tsp sesame seeds
Method
STEP 1
Combine the ponzu, vinegar, ginger and sesame oil in a bowl. Pat the tofu dry using kitchen paper and tear into chunks, then gently toss in the ponzu mixture.

STEP 2
Pour some boiling water over the edamame and set aside for 2 mins before draining thoroughly and seasoning with salt.

STEP 3
Divide the quinoa between bowls and top with the edamame, radishes and carrots. Spoon over the tofu and drizzle over the remaining dressing before scattering over the spring onions, seaweed and sesame seeds.


In [7]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)
assistant = client.beta.assistants.retrieve(ASST_ID)

In [52]:
import logging

# Set up logging configuration
logging.basicConfig(level=logging.INFO)
logging.getLogger("httpx").setLevel(logging.WARNING)


def get_response(recipe):
    thread = client.beta.threads.create()
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=recipe,
    )
    run = client.beta.threads.runs.create(
        thread_id=thread.id, assistant_id=assistant.id, instructions=""
    )
    status = None
    logging.info("Waiting for response...")
    while status != "completed":
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        status = run.status
        time.sleep(1)
    logging.info("Response received 🥳")
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    return messages


def get_file_id(messages):
    annotations = (
        messages.model_dump()
        .get("data", [{}])[0]
        .get("content", [{}])[0]
        .get("text", {})
        .get("annotations", [{}])
    )
    if annotations:
        file_id = annotations[0].get("file_path", {}).get("file_id")
    else:
        file_id = None
    return file_id


def download_file(file_id, fp):
    api_response = client.files.with_raw_response.retrieve_content(file_id)
    if api_response.status_code == 200:
        content = api_response.content
        with open(fp, "wb") as f:
            f.write(content)
        logging.info(f"File downloaded successfully to {fp}")

In [43]:
missed = []
for dir in DATA.ls():
    logging.info(f"Processing {dir}")
    recipe = dir / "recipe.txt"
    recipe = recipe.read_text()
    messages = get_response(recipe)
    file_id = get_file_id(messages)
    if file_id is not None:
        download_file(file_id, dir / "gantt.tsv")
    else:
        logging.info("No file produced")
        missed.append(dir)

# save missed to a file
with open("missed.txt", "w") as f:
    f.write("\n".join([str(x) for x in missed]))

INFO:root:Processing ../../data/001
INFO:__main__:Response received 🥳


IndexError: list index out of range